In [1]:
from dask.distributed import Client, LocalCluster
import bokeh
import geopandas as gpd
import scipy
import libpysal
import momepy
import json
from dask import delayed
import glob
import dask
import dask.dataframe as dd
import momepy
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import json
from tqdm import tqdm
from shapely.geometry import Point
import networkx as nx

/tmp/ipykernel_601/2859145375.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [ ]:
daskCluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(daskCluster)

client

In [ ]:
with open('./out/singapore/what_cells_are_in_what_cluster_dict.json', 'r') as f:
    includes_dict = json.load(f)
    
tessellation = gpd.read_parquet("./out/singapore/tessellation.pq")

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

In [ ]:
streets['nID'] = momepy.unique_id(streets)
# buildings['nID'] = momepy.get_network_id(buildings, streets,
#                                                'nID').values

# buildings = buildings.drop_duplicates("uID").drop(columns="index_right")
# tessellation = tessellation.merge(buildings[['uID', 'nID']], on='uID', how='left')

In [ ]:
### Distance Weight
dist200 = libpysal.weights.DistanceBand.from_dataframe(buildings, 200, ids='uID')

In [ ]:
@dask.delayed
def queen_contiguity_load(n1, includes_dict):
    chunk = gpd.read_parquet(f"./out/singapore/expanded_cells_in_chunk_{int(n1)}.pq")
    
    # weights
    sw1 = libpysal.weights.w_subset(momepy.sw_high(k=1, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    sw3 = libpysal.weights.w_subset(momepy.sw_high(k=3, gdf=chunk, ids='uID'), includes_dict[str(n1)]).to_adjlist()
    
    return (sw1, sw3)

In [ ]:
file_pattern = "./out/singapore/expanded_cells_in_chunk_*.pq"
file_list = glob.glob(file_pattern)
num_files = len(file_list)

# weight_out = [process(n1, includes_dict) for n1 in tqdm(range(num_files))]

weight_out = [queen_contiguity_load(n1, includes_dict) for n1 in range(num_files)]

weight_out = dask.compute(weight_out)


In [ ]:
sw1 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][0] for i in range(num_files)))

sw3 = libpysal.weights.W.from_adjlist(pd.concat(weight_out[0][i][1] for i in range(num_files)))

In [ ]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# new_buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [ ]:
### Building Area
buildings['area'] = momepy.Area(buildings).series

## Building Shapes

### Circular Compactness
buildings['circular_com'] =  momepy.CircularCompactness(buildings).series

### Elongation
blg_elongation = momepy.Elongation(buildings)
buildings['elongation'] = blg_elongation.series

## Squareness
blg_squareness = momepy.Squareness(buildings)
buildings['squareness'] = blg_squareness.series

buildings['eri'] = momepy.EquivalentRectangularIndex(buildings).series

buildings['area'] = momepy.Area(buildings).series


buildings['orientation'] = momepy.Orientation(buildings).series

In [ ]:
tessellation["convexity"] = momepy.Convexity(tessellation).series
tessellation["area"] = momepy.Area(tessellation).series

tessellation["neighbours"] = momepy.Neighbors(tessellation, sw3,'uID').series
tessellation["neighbours400"] = momepy.Neighbors(tessellation, dist200, 'uID').series
tessellation["covered_area"] = momepy.CoveredArea(tessellation, sw1, "uID", verbose=False).series

areas = momepy.Area(tessellation).series
tessellation["mean_area"] = momepy.AverageCharacter(
tessellation, values=areas, spatial_weights=sw3, unique_id='uID').mean

tessellation["mean_area_iq"] = momepy.AverageCharacter(
    tessellation, areas, sw3, 'uID', rng=(25, 75)).mean

tessellation["area_rng"] = momepy.Range(tessellation, values='area',
                                        spatial_weights=sw3, unique_id='uID').series
tessellation["area_iqr"] = momepy.Range(tessellation, values='area',
                        spatial_weights=sw3, unique_id='uID',
                        rng=(25, 75)).series
tessellation["rea_theil"] = momepy.Theil(tessellation, values='area',
                        spatial_weights=sw3,
                        unique_id='uID').series

tessellation["shannon"] = momepy.Shannon(tessellation, values='area',spatial_weights=sw3, unique_id='uID').series


In [ ]:
edg_lin = momepy.Linearity(streets)
streets['linearity'] = edg_lin.series

profile = momepy.StreetProfile(streets, buildings)
streets["width"] = profile.w
streets["width_deviation"] = profile.wd
streets["openness"] = profile.o

In [ ]:
# primal = momepy.gdf_to_nx(streets, approach='primal')

In [ ]:
%%time
# momepy.node_degree(primal)

In [ ]:
%%time
# primal = momepy.closeness_centrality(primal, radius=400, name='closeness400', distance='mm_len', weight='mm_len')

In [ ]:
%%time
# primal = momepy.meshedness(primal, radius=400, distance="mm_len")

In [ ]:
 %%time
primal = momepy.straightness_centrality(primal)

In [ ]:
# %%time
# primal = momepy.closeness_centrality(primal, name='closeness_global', weight='mm_len')

In [ ]:
# %%time
# primal =  momepy.betweenness_centrality(primal, name='betweenness_metric_n', mode='nodes', weight='mm_len', k=100)

In [ ]:
# %%time
# primal = momepy.straightness_centrality(primal)
# # momepy.mean_nodes(primal, 'straightness')
# momepy.mean_nodes(primal, 'closeness400')
# # momepy.mean_nodes(primal, 'closeness_global')
# # momepy.mean_nodes(primal, 'betweenness_metric_n')

# nodes, streets = momepy.nx_to_gdf(primal)

In [ ]:
# dual = momepy.gdf_to_nx(streets, approach='dual')

# dual = momepy.closeness_centrality(dual, radius=5, name='angcloseness400', weight='angle')

# # dual = momepy.closeness_centrality(dual, name='closeness_global_ang', weight='angle')

# # dual = momepy.betweenness_centrality(dual, name='angbetweenness', mode='nodes', weight='angle')

# edges_d = momepy.nx_to_gdf(dual)


In [ ]:
# # Find the uIDs in tessellation that are not in buildings
# tessellation_uids = set(tessellation['uID'])
# buildings_uids = set(buildings['uID'])
# new_uids = tessellation_uids - buildings_uids

# # Create a new GeoDataFrame with the new uIDs and empty geometry
# new_rows = pd.DataFrame({'uID': list(new_uids)})
# new_rows['geometry'] = None
# new_gdf = gpd.GeoDataFrame(new_rows, crs=buildings.crs)

# # Append the new GeoDataFrame to buildings
# buildings = pd.concat([buildings, new_gdf], ignore_index=True)

In [ ]:
circular_compactness = momepy.CircularCompactness(buildings)
buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, sw3, 'uID', momepy.Area(buildings).series).series

buildings['neighbour_dist'] = momepy.NeighborDistance(buildings, sw1, 'uID').series

buildings['interbuilding_distance'] = momepy.MeanInterbuildingDistance(buildings, sw1, 'uID', 3, verbose=False).series

interblg_distance = momepy.MeanInterbuildingDistance(
buildings, sw1, 'uID', spatial_weights_higher=sw3)
buildings['mean_ib_dist'] = interblg_distance.series

buildings['mean_ib_dist'] = momepy.MeanInterbuildingDistance(
    buildings, sw1, 'uID', order=3).series


buildings_net = buildings.dropna(subset=["nID"])

str_align = momepy.StreetAlignment(buildings_net, streets,
                                   'orientation', 'nID',
                                   'nID')
buildings_net['str_align'] = str_align.series

blg_cell_align = momepy.CellAlignment(buildings, tessellation,
                                      'orientation', 'orientation',
                                      'uID', 'uID')
buildings['cell_align'] = blg_cell_align.series


tessellation['car'] = momepy.AreaRatio(tessellation, buildings, 'area', 'area', 'uID').series

buildings['weighted_circom'] = momepy.WeightedCharacter(
buildings, circular_compactness.series, sw3, 'uID', momepy.Area(buildings).series).series



In [ ]:
merged = tessellation.merge(buildings.drop(columns=['nID', 'geometry']), on='uID')
merged = merged.merge(streets.drop(columns='geometry'), on='nID', how='left')
merged = merged.merge(nodes.drop(columns='geometry'), on='nodeID', how='left')
merged = merged.merge(edges_d.drop(columns='geometry'), on='nodeID', how='left')
